In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [55]:
time_space=np.linspace(0,100,1000)
ecg_signal=np.sin(time_space)+np.random.normal(scale=0.1,size=1000)
df=pd.DataFrame({'ECG':ecg_signal})

In [54]:
sequence_length=20
def create_sequences(data,sequence_length):
    sequences=[]
    targets=[]
    for i in range(len(data)-sequence_length):
        sequence=data[i:i+sequence_length]
        target=data[i+sequence_length]
        sequences.append(sequence)
        targets.append(target)
    return np.array(sequences),np.array(targets)
sequences,targets=create_sequences(df['ECG'].values,sequence_length)
X=torch.tensor(sequences,dtype=torch.float32).unsqueeze(-1)
y=torch.tensor(targets,dtype=torch.float32).unsqueeze(-1)
print(X.shape,y.shape)

torch.Size([980, 20, 1]) torch.Size([980, 1])


In [64]:
class RNNModel(nn.Module):
    def __init__(self,input_size=1,hidden_size=64,num_layers=2):
        super(RNNModel,self).__init__()
        self.rnn=nn.RNN(input_size,hidden_size,num_layers,batch_first=True)
        self.fc=nn.Linear(hidden_size,1)
    def forward(self,x):
        rnn_out,_=self.rnn(x)
        output=self.fc(rnn_out[:,-1,:])
        return output
model=RNNModel()
print(model)

RNNModel(
  (rnn): RNN(1, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [68]:
criteraion =nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)
epochs=50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs=model(X)
    loss=criteraion(outputs,y)
    loss.backward()
    optimizer.step()
    if((epoch+1)%10)==0:
        print(epoch+1, loss.item())

10 0.026969023048877716
20 0.015439685434103012
30 0.012836609967052937
40 0.012370340526103973
50 0.011758217588067055


In [75]:
model.eval()
with torch.no_grad():
    test_seq=X[-1].unsqueeze(0)
    print(test_seq)
    preditions=[]
    for i in range(50):
        pred=model(test_seq)
        preditions.append(pred.item())
        print(i+1,pred.item())
        pred=pred.unsqueeze(-1)
        test_seq=torch.cat((test_seq[:,1:,:],pred),dim=1)

tensor([[[-0.5386],
         [-0.5041],
         [-0.5947],
         [-0.7555],
         [-0.9127],
         [-0.8045],
         [-1.0462],
         [-0.8447],
         [-1.0489],
         [-0.8571],
         [-0.7741],
         [-0.7616],
         [-1.1112],
         [-1.0200],
         [-0.9744],
         [-0.7306],
         [-0.6324],
         [-0.7331],
         [-0.8680],
         [-0.5309]]])
1 -0.5158066749572754
2 -0.4180980324745178
3 -0.3667689561843872
4 -0.2727890610694885
5 -0.1389951854944229
6 -0.055965691804885864
7 0.036437779664993286
8 0.15855278074741364
9 0.25879406929016113
10 0.35560595989227295
11 0.45310914516448975
12 0.543912947177887
13 0.6281811594963074
14 0.7026522755622864
15 0.769114077091217
16 0.8289405703544617
17 0.8791115283966064
18 0.9199994802474976
19 0.952786922454834
20 0.9766440391540527
21 0.991509199142456
22 0.9988029599189758
23 0.997380793094635
24 0.9867123961448669
25 0.966997504234314
26 0.9379997253417969
27 0.8996660113334656
28 0.